# GPT1 - Improving Language Understanding by Generative Pre-Training

## Resumen

 * Entrenan el modelo en un gran corpus de texto sin supervisión. Modelado del lenguaje. Aprender un modelo de lenguaje de alta capacidad en un gran corpus de texto
 * Luego hacen fine-tuning en tareas de NLP supervisadas. Es decir con datasets etiquetados. Ajuste fino en una tarea objetivo con supervisión. Ajuste fino, en la que adaptamos el modelo a una tarea discriminativa con datos etiquetados. CUando evaluan al modelo en la tarea supervisada, no solo le evaluan por esa tarea, sino por lo bien que predice el siguiente token, esto ayuda a mejorar la generalización del modelo supervisado y hace que el modelo converja más rápido.
 * Utilizan la arquitectura transformer, que mejora al uso de RNN en que lo aprendido en el primer entrenamiento es más fácil de transferir a tareas supervisadas.
 * Evaluan el modelo en cuatro tipos de tareas de comprensión del lenguaje:
    * Inferencia del lenguaje natural
    * Respuesta a preguntas
    * Similitud semántica
    * Clasificación de textos. 
 * El modelo, agnóstico de tareas generales, supera a los modelos entrenados discriminativamente que emplean arquitecturas diseñadas específicamente para cada tarea, mejorando significativamente el estado del arte en 9 de las 12 tareas estudiadas.
 * También analizamos los comportamientos de "disparo cero" del modelo preentrenado en cuatro entornos diferentes y demostramos que adquiere un conocimiento lingüístico útil para las tareas posteriores.
 * En los últimos años, los investigadores han demostrado los beneficios de utilizar incrustaciones de palabras [11, 39, 42], que se entrenan en corpus no etiquetados, para mejorar el rendimiento en una variedad de tareas [8, 11, 26, 45]. Sin embargo, estos enfoques transfieren principalmente información a nivel de palabra, mientras que nosotros pretendemos capturar la semántica de nivel superior.
 * Utilizamos un decodificador Transformer multicapa
 * 

¿Cómo es la arquitectura? ¿Qué pasa con la entrada del encoder?

## Generación de texto

Primero hay que instalar `ftfy` y `spacy` mediante

```bash
pip install ftfy spacy
```

Una vez instaladas, debes descargar el modelo de lenguaje de spacy que deseas utilizar. Por ejemplo, para descargar el modelo de inglés, puedes ejecutar:

```bash
python -m spacy download en_core_web_sm
```

Para generar texto vamos a utilizar el modelo desde el repositorio de [GPT1](https://huggingface.co/openai-community/openai-gpt) de Hugging Face.

Importamos las librerías

In [1]:
import torch
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel, AutoTokenizer

Si te fijas hemos importado `OpenAIGPTTokenizer` y `AutoTokenizer`. Esto es porque en la [model card](https://huggingface.co/openai-community/openai-gpt) de GPT1 se indica que se use `OpenAIGPTTokenizer`, pero en el post de la librería [transformers](https://maximofn.com/hugging-face-transformers/) explicamos que se debe usar `AutoTokenizer` para cargar el tokenizador. Así que vamos a probar los dos

In [2]:
ckeckpoints = "openai-community/openai-gpt"
tokenizer = OpenAIGPTTokenizer.from_pretrained(ckeckpoints)
auto_tokenizer = AutoTokenizer.from_pretrained(ckeckpoints)

input_tokens = tokenizer("Hello, my dog is cute and", return_tensors="pt")
input_auto_tokens = auto_tokenizer("Hello, my dog is cute and", return_tensors="pt")

print(f"input tokens: \n{input_tokens}")
print(f"input auto tokens: \n{input_auto_tokens}")

input tokens: 
{'input_ids': tensor([[3570,  240,  547, 2585,  544, 4957,  488]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
input auto tokens: 
{'input_ids': tensor([[3570,  240,  547, 2585,  544, 4957,  488]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


Como se puede ver con los dos tokenizadores se obtienen los mismos tokens. Así que para que el código sea más general, de manera que si se cambian los ckeckpoints no haya que cambiar el código, vamos a utilizar `AutoTokenizer`

Creamos entonces el device, el tokenizador y el modelo

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(ckeckpoints)
model = OpenAIGPTLMHeadModel.from_pretrained(ckeckpoints).to(device)

Creamos los tokens de entrada al modelo

In [4]:
input_sentence = "Hello, my dog is cute and"
input_tokens = tokenizer(input_sentence, return_tensors="pt").to(device)

input_tokens

{'input_ids': tensor([[3570,  240,  547, 2585,  544, 4957,  488]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Se los pasamos al modelo para generar los tokens de salida

In [5]:
output_tokens = model.generate(**input_tokens)

print(f"output tokens: \n{output_tokens}")

output tokens: 
tensor([[ 3570,   240,   547,  2585,   544,  4957,   488,   249,   719,   797,
           485,   921,   575,   562,   246,  1671,   239,   244, 40477,   244]],
       device='cuda:0')


/home/wallabot/miniconda3/envs/nlp/lib/python3.11/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Decodificamos los tokens para obtener la sentencia de salida

In [6]:
decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f"decoded output: \n{decoded_output}")

decoded output: 
hello, my dog is cute and i'm going to take him for a walk. " 
 "


Ya hemos conseguido generar texto con GPT1

### Generar texto token a token

#### Greedy search

Hemos usado `model.generate` para generar los tokens de salida de golpe, pero vamos a ver cómo generarlos uno a uno. Para ello, en vez de usar `model.generate` vamos a usar `model`, que en realidad lo que hace es llamar al método `model.forward`

In [7]:
outputs = model(**input_tokens)

outputs

CausalLMOutput(loss=None, logits=tensor([[[ -5.9486,  -5.8697, -18.4258,  ...,  -9.7371, -10.4495,   0.8814],
         [ -6.1212,  -4.8031, -14.3970,  ...,  -6.5411,  -9.5051,  -1.2015],
         [ -7.4231,  -6.3615, -14.7297,  ..., -10.4575,  -8.4600,  -1.5183],
         ...,
         [ -5.4751,  -5.8803, -13.7767,  ..., -10.5048, -12.4167,  -6.1584],
         [ -7.2052,  -6.0198, -21.5040,  ..., -16.2941, -14.0494,  -1.2416],
         [ -7.7240,  -7.3631, -17.3174,  ..., -12.1546, -12.3327,  -1.7169]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>), hidden_states=None, attentions=None)

Vemos que saca muchos datos, primero vamos a ver las keys de la salida

In [8]:
outputs.keys()

odict_keys(['logits'])

En este caso solo tenemos los logits del modelo, vamos a ver su tamaño

In [9]:
logits = outputs.logits

logits.shape

torch.Size([1, 7, 40478])

Vamos a ver cuantos tokens teníamos a la entrada

In [10]:
input_tokens.input_ids.shape

torch.Size([1, 7])

Vaya, a la salida tenemos el mismo número de logits que a la entrada. Esto es normal

Obtenemos los logits de la última posición de la salida

In [11]:
nex_token_logits = logits[0,-1]

nex_token_logits.shape

torch.Size([40478])

Hay un total de 40478 logits, es decir, hay un vocabulario de 40478 tokens y tenemos que ver cuál es el token con mayor probabilidad, para ello primero calculamos las softmax

In [12]:
softmax_logits = torch.softmax(nex_token_logits, dim=0)

softmax_logits.shape

torch.Size([40478])

In [13]:
next_token_prob, next_token_id = torch.max(softmax_logits, dim=0)

next_token_prob, next_token_id

(tensor(0.1898, device='cuda:0', grad_fn=<MaxBackward0>),
 tensor(249, device='cuda:0'))

In [14]:
tokenizer.decode(next_token_id.item())

'i'

Hemos obtenido el siguiente token mediante el método greedy, es decir, el token con mayor probabilidad. Pero ya vimos en el post de la librería transformers, las [formas de generar textos](https://maximofn.com/hugging-face-transformers/#Formas-de-generaci%C3%B3n-de-texto) que se puede hacer sampling, top-k, top-p, etc.

Vamos a meter todo en una función y ver qué sale si generamos unos cuantos tokens

In [15]:
def generate_next_greedy_token(input_sentence, tokenizer, model, device):
    input_tokens = tokenizer(input_sentence, return_tensors="pt").to(device)
    outputs = model(**input_tokens)
    logits = outputs.logits
    nex_token_logits = logits[0,-1]
    softmax_logits = torch.softmax(nex_token_logits, dim=0)
    next_token_prob, next_token_id = torch.max(softmax_logits, dim=0)
    return next_token_prob, next_token_id

In [16]:
def generate_greedy_text(input_sentence, tokenizer, model, device, max_length=20):
    generated_text = input_sentence
    for _ in range(max_length):
        next_token_prob, next_token_id = generate_next_greedy_token(generated_text, tokenizer, model, device)
        generated_text += tokenizer.decode(next_token_id.item())
    return generated_text

Ahora generamos texto

In [17]:
generate_greedy_text("Hello, my dog is cute and", tokenizer, model, device)

'Hello, my dog is cute andi."\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

#### Sampling con temperatura, top-k y top-p

Primero vamos a recordar cómo se haría con transformers

In [18]:
input_tokens = tokenizer(input_sentence, return_tensors="pt").to("cuda")
tokens_output = model.generate(**input_tokens, max_new_tokens=500, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)
sentence_output = tokenizer.decode(tokens_output[0], skip_special_tokens=True)

print(sentence_output)

hello, my dog is cute and i love him, but i'm not having sex with you right now. " 
 he laughs and takes his hand off my thigh. " you're such a liar, baby. " 
 " i'm not lying, " i tell him. 
 " you're not? " 
 " nope. " 
 " i think i'm going to go to bed, " i announce. 
 " i 'll see you in the morning. " 
 i stand, grab my bag and purse and leave the room, closing the door behind me. 
 i'm not a liar. 
 i'm not a liar. 
 i am not a liar. 
 i'm not a liar. 
 i'm not a liar. 
 i'm not a liar. 
 i'm a liar. 
 i'm a fucking liar. 
 i'm a fucking liar. 
 i'm a fucking liar. 
 i'm a fucking liar. 
 i'm a fucking a * * hole. 
 i'm a f * * king a * * hole. 
 i'm a f * * king a * * hole. 
 i'm a f * * king a * * hole. 
 " hey, babe, " i call out, walking into the kitchen. 
 " hey, " she responds, turning to face me. 
 " what's up? " i ask, sitting down at the island. 
 " i just wanted to see if you 'd like to do something tonight. " 
 " what do you mean? " 
 " i thought you might want to get t

Ahora vamos a hacerlo token a token

In [19]:
import torch
import torch.nn.functional as F

def generate_next_token_sampling(input_sentence, tokenizer, model, device, temperature, top_p, top_k):
    input_tokens = tokenizer(input_sentence, return_tensors="pt").to(device)
    outputs = model(**input_tokens)
    logits = outputs.logits
    next_token_logits = logits[0, -1] / temperature  # Apply temperature scaling
    
    # Calculate top-k and top-p filtering
    top_k_logits, top_k_indices = torch.topk(next_token_logits, k=top_k, dim=0)
    next_token_logits = top_k_logits
    next_token_indices = top_k_indices
    
    # Calculate probabilities with softmax
    softmax_logits = F.softmax(next_token_logits, dim=0)
    
    # Apply top-p filtering
    cumulative_probs = torch.cumsum(softmax_logits, dim=0)
    sorted_indices = torch.argsort(cumulative_probs, descending=True)
    top_p_indices = sorted_indices[cumulative_probs[sorted_indices] <= top_p]
    next_token_probs = softmax_logits[top_p_indices]
    next_token_indices = next_token_indices[top_p_indices]
    
    # Sample from the filtered probabilities
    next_token_prob = torch.multinomial(next_token_probs, num_samples=1)
    next_token_id = next_token_indices[next_token_prob]
    
    return next_token_prob, next_token_id

In [20]:
def generate_sampling_text(input_sentence, tokenizer, model, device, temperature=1.0, top_p=0.9, top_k=50, max_length=20):
    generated_text = input_sentence
    for _ in range(max_length):
        next_token_prob, next_token_id = generate_next_token_sampling(generated_text, tokenizer, model, device, temperature, top_p, top_k)
        generated_text += tokenizer.decode(next_token_id.item())
    return generated_text

Generamos 100 tokens

In [21]:
# generate_sampling_text("Hello, my dog is cute and", tokenizer, model, device, temperature=0.7, top_p=0.95, top_k=50, max_length=100)

'Hello, my dog is cute andhe\'sgotaandhe\'-idonot,"hesaid,shakinghisheadinthecollar.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

## Fine tuning GPT

### Cálculo de la loss

Antes de empezar a hacer el fine tuning de GPT1 vamos a ver una cosa. Antes cuando obteníamos la salida del modelo hacíamos esto

In [22]:
outputs = model(**input_tokens)

outputs

CausalLMOutput(loss=None, logits=tensor([[[ -5.9486,  -5.8697, -18.4258,  ...,  -9.7371, -10.4495,   0.8814],
         [ -6.1212,  -4.8031, -14.3970,  ...,  -6.5411,  -9.5051,  -1.2015],
         [ -7.4231,  -6.3615, -14.7297,  ..., -10.4575,  -8.4600,  -1.5183],
         ...,
         [ -5.4751,  -5.8803, -13.7767,  ..., -10.5048, -12.4167,  -6.1584],
         [ -7.2052,  -6.0198, -21.5040,  ..., -16.2941, -14.0494,  -1.2416],
         [ -7.7240,  -7.3631, -17.3174,  ..., -12.1546, -12.3327,  -1.7169]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>), hidden_states=None, attentions=None)

Se puede ver que obtenemos `loss=None`

In [23]:
print(outputs.loss)

None


Como vamos a necesitar la loss para hacer el fine tuning, vamos a ver cómo obtenerla.

Si nos vamos a la documentación del método [forward](https://huggingface.co/docs/transformers/model_doc/openai-gpt#transformers.OpenAIGPTLMHeadModel.forward) de `OpenAIGPTLMHeadModel`, podemo ver que dice que a la salida devuelve un objeto de tipo `transformers.modeling_outputs.CausalLMOutput`, así que si nos vamos a la documentación de [transformers.modeling_outputs.CausalLMOutput](https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/output#transformers.modeling_outputs.CausalLMOutput), podemos ver que dice que devuelve `loss` si se le pasa `labels` al método `forward`.

Si nos vamos a la fuente del código del método [forward](https://github.com/huggingface/transformers/blob/main/src/transformers/models/openai/modeling_openai.py#L544), vemos este bloque de código

```python
        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
```

Es decir, la `loss` se calcula de la siguiente manera

 * Shift de logits y labels: La primera parte es desplazar los logits (`lm_logits`) y las etiquetas (`labels`) para que los `tokens < n` predigan `n`, es decir, desde una posición `n` se predice el siguiente token a partir de los anteriores.
 * CrossEntropyLoss: Se crea una instancia de la función de pérdida `CrossEntropyLoss()`.
 * Flatten tokens: A continuación, se aplanan los logits y las etiquetas utilizando `view(-1, shift_logits.size(-1))` y `view(-1)`, respectivamente. Esto se hace para que los logits y las etiquetas tengan la misma forma para la función de pérdida.
 * Cálculo de la pérdida: Finalmente, se calcula la pérdida utilizando la función de pérdida `CrossEntropyLoss()` con los logits aplanados y las etiquetas aplanadas como entradas.

En resumen, la `loss` se calcula como la pérdida de entropía cruzada entre los logits desplazados y aplanados y las etiquetas desplazadas y aplanadas.

Por tanto, si al método `forward` le pasamos los labels, nos devolverá la `loss`

In [24]:
outputs = model(**input_tokens, labels=input_tokens.input_ids)

outputs.loss

tensor(4.2607, device='cuda:0', grad_fn=<NllLossBackward0>)

### Dataset

Para el etrenamiento vamos a usar un dataset de chistes en inglés [short-jokes-dataset](https://huggingface.co/datasets/Maximofn/short-jokes-dataset), que es un dataset con 232 mil chistes en inglés.

Descargamos el dataset

In [25]:
from datasets import load_dataset

jokes = load_dataset("Maximofn/short-jokes-dataset")
jokes

DatasetDict({
    train: Dataset({
        features: ['ID', 'Joke'],
        num_rows: 231657
    })
})

Vamos a verlo un poco

In [26]:
jokes["train"][0]

{'ID': 1,
 'Joke': '[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"'}

### Entrenamiento con Pytorch

Primero vamos a ver cómo se haría el entrenamiento con puro Pytorch

In [1]:
import torch
from transformers import OpenAIGPTLMHeadModel, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ckeckpoints = "openai-community/openai-gpt"
tokenizer = AutoTokenizer.from_pretrained(ckeckpoints)
model = OpenAIGPTLMHeadModel.from_pretrained(ckeckpoints)

special_tokens_dict = {'pad_token': '[PAD]'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

In [2]:
from datasets import load_dataset

jokes = load_dataset("Maximofn/short-jokes-dataset")

In [14]:
import tqdm

progress = tqdm.tqdm(jokes["train"])

max_length = 0
for joke in progress:
    joke_text = joke["Joke"]
    input_tokens = tokenizer(joke_text, return_tensors="pt")
    len_input_tokens = input_tokens.input_ids.shape[1]
    if len_input_tokens > max_length:
        max_length = len_input_tokens
    progress.set_postfix({"max_length": max_length})

100%|██████████| 231657/231657 [02:41<00:00, 1435.09it/s, max_length=126]


In [15]:
max_dataset_length = max_length
max_dataset_length

126

In [3]:
max_dataset_length = 126

#### Pytorch dataset

Creamos una clase dataset de Pytorch

La instanciamos

Vemos un ejemplo

### Dataloader

Creamos ahora un dataloader de Pytorch

Vemos un batch

### Training

In [4]:
from torch.utils.data import Dataset

class JokesDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_seq_len=400):
        self.dataset = dataset
        self.joke = "JOKE: "
        self.end_of_text_token = "<|endoftext|>"
        self.tokenizer = tokenizer
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.max_seq_len = max_seq_len
        self.vocab_size = len(tokenizer)
        
    def __len__(self):
        return len(self.dataset["train"])

    def __getitem__(self, item):
        sentence = self.joke + self.dataset["train"][item]["Joke"] + self.end_of_text_token
        tokens = self.tokenizer(sentence, return_tensors="pt", max_length=self.max_seq_len, padding="max_length", truncation=True)
        tokens.input_ids = tokens.input_ids.clamp(0, self.vocab_size - 1)
        return sentence, tokens

In [5]:
dataset = JokesDataset(jokes, tokenizer=tokenizer, max_seq_len=max_dataset_length+2)

In [6]:
sentence, tokens = dataset[5]
print(sentence)
tokens.input_ids.shape, tokens.attention_mask.shape

JOKE: Why can't Barbie get pregnant? Because Ken comes in a different box. Heyooooooo<|endoftext|>


(torch.Size([1, 128]), torch.Size([1, 128]))

In [7]:
from torch.utils.data import DataLoader

BS = 80
joke_dataloader = DataLoader(dataset, batch_size=BS, shuffle=True)

In [8]:
sentences, tokens = next(iter(joke_dataloader))
len(sentences), tokens.input_ids.shape, tokens.attention_mask.shape

(80, torch.Size([80, 1, 128]), torch.Size([80, 1, 128]))

In [9]:
from transformers import AdamW
import tqdm

EPOCHS = 1
LEARNING_RATE = 3e-7

model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)

    progress_bar = tqdm.tqdm(joke_dataloader, desc="Training")
    
    for batch in progress_bar:

        sentences, tokens = batch
        tokens = tokens.to(device)
        
        outputs = model(tokens.input_ids.squeeze(), labels=tokens.input_ids.squeeze())
        loss = outputs.loss                        
        loss_value = loss.item()
        loss.backward()
                    
        optimizer.step()
        optimizer.zero_grad()

        progress_bar.set_postfix({'training_loss': loss_value})
        # break
    # break

/home/wallabot/miniconda3/envs/nlp/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


EPOCH 0 started==============================


Training:  10%|▉         | 285/2896 [02:28<22:42,  1.92it/s, training_loss=1.56]


KeyboardInterrupt: 

### Inference

In [14]:
sentence_joke = "JOKE:"
input_tokens_joke = tokenizer(sentence_joke, return_tensors="pt").to(device)
output_tokens_joke = model.generate(**input_tokens_joke)
decoded_output_joke = tokenizer.decode(output_tokens_joke[0], skip_special_tokens=True)

print(f"decoded joke: \n{decoded_output_joke}")

decoded joke: 
joke : " i'm not sure i can do this. " 
 " you can do this,
